In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import bisect
import glob
import os
import re
import time

import torch
import torchvision
import numpy as np

import pytorch_mask_rcnn as pmr

from tqdm import tqdm

In [3]:
from torch.utils.data import Dataset

class StudentDataset(Dataset):
    def __init__(self, dataset_dir, labeled = True):
        self.dataset_dir = dataset_dir
        self.labeled = labeled
        video_dirs = os.listdir(dataset_dir)
        self.images = []
        for video_dir in video_dirs:
            video_dir = os.path.join(dataset_dir, video_dir)
            files = os.listdir(video_dir)
            image_ids = [int(f[6:-4]) for f in files if f.endswith('png')]
            image_files = [os.path.join(video_dir, f) for f in files if f.endswith('png')]
            mask_file = os.path.join(video_dir, 'mask.npy')
            self.images += [(f, i, mask_file) for f, i in zip(image_files, image_ids)]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        filename, i, mask_file = self.images[idx]
        image = torchvision.io.read_image(filename).float()
        if self.labeled:
            mask = np.load(mask_file)[i]
            objs = set(mask.reshape(-1)) - {0}
            boxes = []
            labels = []
            masks = []
            for obj in objs:
                cur_mask = mask == obj
                masks.append(cur_mask)
                labels.append(obj)
                rows = np.any(cur_mask, axis=0)
                cols = np.any(cur_mask, axis=1)
                xmin, xmax = np.where(rows)[0][[0, -1]]
                ymin, ymax = np.where(cols)[0][[0, -1]]
                boxes.append([xmin, ymin, xmax, ymax])
            target = {
                'boxes': torch.FloatTensor(boxes),
                'labels': torch.LongTensor(labels),
                'masks': torch.ByteTensor(masks)
            }
            return (image, target)
        else:
            return image

In [4]:
train_dataset = StudentDataset('Dataset_Student/train/')
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True)
val_dataset = StudentDataset('Dataset_Student/val/')
val_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=False)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [35]:
class AttrDict(dict):
    def __getattr__(self, key):
        return self[key]

    def __setattr__(self, key, value):
        self[key] = value

args = AttrDict()

args.warmup_iters = 1
args.print_freq = 100
args.lr = 1e-6
args.epochs = 20
args.momentum = 0.9
args.weight_decay = 0.0001
args.lr_steps = [6, 7]
args.ckpt_path = 'maskrcnn_coco-6-11.pth'

In [36]:
num_classes = 49
model = pmr.maskrcnn_resnet50(False, num_classes, False).to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    params, lr=args.lr, momentum=args.momentum, weight_decay=args.weight_decay)
lr_lambda = lambda x: 0.1 ** bisect.bisect(args.lr_steps, x)

start_epoch = 0

if args.ckpt_path is not None:
    checkpoint = torch.load(args.ckpt_path, map_location=device)
    model.load_state_dict(checkpoint['model'])
    optimizer.load_state_dict(checkpoint['optimizer'])

    start_epoch = checkpoint['epochs'] + 1

C:\ProgramData\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
for epoch in range(start_epoch, args.epochs):
    print("\nepoch: {}".format(epoch + 1))

    args.lr_epoch = args.lr
    print("lr_epoch: {:.5f}, factor: {:.5f}".format(args.lr_epoch, lr_lambda(epoch)))
    iter_train = pmr.train_one_epoch(model, optimizer, train_loader, device, epoch, args)

    pmr.save_ckpt(model, optimizer, epoch, args.ckpt_path) #, eval_info=str(eval_output))

    model.eval()
    with torch.no_grad():
        total_ious = []
        for image, target in tqdm(val_loader, leave=False):
            image = image[0].to(device)
            res = model(image)
            target_masks = {int(label): mask.to(device) for label, mask in zip(target['labels'][0], target['masks'][0])}
            res_masks = {int(label): torch.zeros_like(mask).to(device) for label, mask in target_masks.items()}
            if len(res['labels']) > 0:
                for label, mask in zip(res['labels'], res['masks'] > 0.5):
                    label = int(label)
                    if label in res_masks:
                        res_masks[label] = res_masks[label] | mask
            ious = []
            for label in target_masks:
                iou = 0
                if label in res_masks:
                    target_mask = target_masks[label]
                    res_mask = res_masks[label]
                    iou = float((target_mask & res_mask).sum() / (target_mask | res_mask).sum())
                ious.append(iou)
            iou = np.mean(ious)
            total_ious.append(iou)
        total_iou = np.mean(total_ious)
    print('Epoch', epoch, 'IOU on validataion set:', total_iou)


epoch: 8
lr_epoch: 0.00000, factor: 0.01000


C:\Users\Administrator\AppData\Local\Temp\ipykernel_60268\1903523856.py:41: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  'masks': torch.ByteTensor(masks)


154000	 0.014	0.019	0.124	0.095	0.609
154100	 0.019	0.055	0.102	0.079	0.606
154200	 0.010	0.023	0.050	0.052	0.456
154300	 0.037	0.079	0.072	0.062	0.461
154400	 0.024	0.103	0.076	0.039	0.665
154500	 0.025	0.073	0.062	0.027	0.429
154600	 0.005	0.009	0.014	0.018	0.304
154700	 0.012	0.036	0.068	0.021	0.446
154800	 0.047	0.041	0.087	0.061	0.681
154900	 0.029	0.063	0.083	0.084	0.599
155000	 0.011	0.033	0.084	0.048	0.484
155100	 0.116	0.195	0.182	0.126	0.495
155200	 0.034	0.093	0.140	0.085	0.499
155300	 0.064	0.171	0.048	0.024	0.674
155400	 0.010	0.013	0.028	0.044	0.799
155500	 0.024	0.068	0.052	0.025	0.613
155600	 0.009	0.006	0.038	0.024	0.316
155700	 0.047	0.203	0.133	0.072	0.559
155800	 0.019	0.051	0.060	0.041	0.458
155900	 0.019	0.061	0.104	0.074	0.355
156000	 0.017	0.060	0.052	0.033	0.675
156100	 0.020	0.060	0.110	0.060	0.443
156200	 0.022	0.046	0.063	0.044	0.549
156300	 0.027	0.040	0.088	0.084	0.681
156400	 0.071	0.100	0.086	0.072	0.660
156500	 0.013	0.050	0.087	0.069	0.613
156600	 0.02

Epoch 7 IOU on validataion set: 0.3398456461768084

epoch: 9
lr_epoch: 0.00000, factor: 0.01000
176000	 0.015	0.039	0.066	0.047	0.533
176100	 0.022	0.020	0.100	0.071	0.477
176200	 0.012	0.008	0.144	0.136	0.532
176300	 0.016	0.008	0.177	0.165	0.383
176400	 0.010	0.015	0.168	0.133	0.395
176500	 0.024	0.024	0.076	0.043	0.350
176600	 0.011	0.010	0.166	0.146	0.513
176700	 0.012	0.006	0.136	0.138	0.378
176800	 0.007	0.007	0.140	0.077	0.612
176900	 0.006	0.015	0.092	0.055	0.431
177000	 0.014	0.017	0.153	0.123	0.437
177100	 0.005	0.025	0.072	0.042	0.481
177200	 0.007	0.009	0.097	0.047	0.730
177300	 0.026	0.028	0.106	0.082	0.496
177400	 0.079	0.072	0.122	0.104	0.511
177500	 0.036	0.063	0.045	0.023	0.412
177600	 0.010	0.008	0.096	0.067	0.681
177700	 0.023	0.022	0.079	0.088	0.354
177800	 0.019	0.035	0.148	0.144	0.643
177900	 0.008	0.010	0.107	0.096	0.586
178000	 0.016	0.016	0.131	0.083	0.538
178100	 0.041	0.052	0.101	0.063	0.449
178200	 0.016	0.017	0.106	0.051	0.301
178300	 0.018	0.036	0.142	0.10

Epoch 8 IOU on validataion set: 0.44279991963704163

epoch: 10
lr_epoch: 0.00000, factor: 0.01000
198000	 0.028	0.020	0.167	0.161	0.436
198100	 0.011	0.007	0.180	0.169	0.619
198200	 0.037	0.073	0.136	0.084	0.491
198300	 0.012	0.018	0.110	0.090	0.552
198400	 0.079	0.133	0.090	0.072	0.456
198500	 0.006	0.004	0.082	0.097	0.441
198600	 0.010	0.023	0.136	0.143	0.375
198700	 0.019	0.007	0.107	0.073	0.411
198800	 0.004	0.002	0.043	0.037	0.529
198900	 0.028	0.024	0.089	0.066	0.379
199000	 0.025	0.027	0.094	0.060	0.385
199100	 0.014	0.027	0.137	0.117	0.409
199200	 0.012	0.002	0.114	0.063	0.945
199300	 0.006	0.002	0.028	0.031	0.365
199400	 0.011	0.011	0.112	0.096	0.570
199500	 0.031	0.053	0.108	0.100	0.303
199600	 0.024	0.026	0.098	0.060	0.993
199700	 0.015	0.017	0.119	0.080	0.329
199800	 0.011	0.008	0.075	0.073	0.258
199900	 0.015	0.046	0.123	0.144	0.466
200000	 0.021	0.036	0.048	0.035	0.565
200100	 0.016	0.021	0.148	0.108	0.403
200200	 0.034	0.052	0.113	0.069	0.392
200300	 0.009	0.024	0.157	0.

...

Epoch 12 IOU on validataion set: 0.5485094838588945

epoch: 14
lr_epoch: 0.00000, factor: 0.01000
286000	 0.016	0.008	0.158	0.091	0.277
286100	 0.013	0.023	0.095	0.079	0.372
286200	 0.019	0.022	0.166	0.144	0.389
286300	 0.004	0.006	0.117	0.063	0.231
286400	 0.008	0.007	0.060	0.069	0.373
286500	 0.004	0.003	0.099	0.070	0.247
286600	 0.008	0.007	0.090	0.065	0.209
286700	 0.009	0.010	0.061	0.047	0.201
286800	 0.012	0.011	0.081	0.053	0.260
286900	 0.013	0.014	0.080	0.079	0.756
287000	 0.004	0.006	0.053	0.079	0.265
287100	 0.005	0.009	0.042	0.027	0.289
287200	 0.022	0.035	0.068	0.053	0.261
287300	 0.004	0.002	0.037	0.021	0.181
287400	 0.008	0.008	0.101	0.091	0.221
287500	 0.007	0.005	0.060	0.078	0.266
287600	 0.031	0.054	0.079	0.075	0.346
287700	 0.015	0.014	0.141	0.100	0.369
287800	 0.022	0.008	0.090	0.071	0.310
287900	 0.013	0.016	0.133	0.096	0.401
288000	 0.020	0.013	0.130	0.090	0.216
288100	 0.017	0.016	0.096	0.078	0.238
288200	 0.039	0.038	0.116	0.093	0.238
288300	 0.019	0.009	0.089	0.

Epoch 13 IOU on validataion set: 0.5464361155692743

epoch: 15
lr_epoch: 0.00000, factor: 0.01000
308000	 0.006	0.001	0.016	0.004	0.168
308100	 0.027	0.022	0.135	0.093	0.256
308200	 0.023	0.023	0.118	0.107	0.288
308300	 0.034	0.006	0.031	0.018	0.213
308400	 0.019	0.019	0.085	0.068	0.257
308500	 0.007	0.003	0.172	0.082	0.257
308600	 0.010	0.011	0.068	0.085	0.309
308700	 0.009	0.002	0.030	0.048	0.230
308800	 0.009	0.004	0.065	0.053	0.187
308900	 0.021	0.009	0.146	0.146	0.503
309000	 0.005	0.004	0.014	0.015	0.331
309100	 0.034	0.043	0.062	0.044	0.267
309200	 0.005	0.004	0.057	0.037	0.213
309300	 0.007	0.004	0.071	0.035	0.259
309400	 0.013	0.004	0.041	0.068	0.293
309500	 0.031	0.034	0.098	0.115	0.290
309600	 0.015	0.020	0.157	0.102	0.298
309700	 0.020	0.005	0.170	0.106	0.308
309800	 0.011	0.007	0.085	0.114	0.268
309900	 0.008	0.007	0.225	0.121	0.356
310000	 0.040	0.030	0.195	0.148	0.464
310100	 0.092	0.142	0.124	0.072	0.245
310200	 0.009	0.012	0.106	0.100	0.304
310300	 0.020	0.009	0.058	0.

Epoch 14 IOU on validataion set: 0.5632584558336837

epoch: 16
lr_epoch: 0.00000, factor: 0.01000
330000	 0.069	0.059	0.085	0.049	0.233
330100	 0.016	0.024	0.120	0.080	0.319
330200	 0.033	0.011	0.124	0.078	0.366
330300	 0.019	0.027	0.163	0.126	0.361
330400	 0.015	0.018	0.065	0.084	0.455
330500	 0.005	0.002	0.030	0.037	0.235
330600	 0.010	0.012	0.050	0.029	0.154
330700	 0.023	0.023	0.089	0.069	0.331
330800	 0.011	0.003	0.061	0.062	0.205
330900	 0.041	0.023	0.123	0.072	0.459
331000	 0.008	0.009	0.092	0.061	0.251
331100	 0.008	0.002	0.028	0.016	0.191
331200	 0.016	0.007	0.138	0.122	0.431
331300	 0.017	0.029	0.072	0.036	0.271
331400	 0.015	0.029	0.103	0.037	0.205
331500	 0.010	0.007	0.116	0.120	0.245
331600	 0.005	0.004	0.095	0.063	0.208
331700	 0.004	0.002	0.036	0.029	0.180
331800	 0.009	0.002	0.066	0.030	0.231
331900	 0.026	0.005	0.125	0.122	0.375
332000	 0.007	0.012	0.110	0.062	0.461
332100	 0.032	0.021	0.142	0.077	0.400
332200	 0.006	0.006	0.107	0.099	0.345
332300	 0.033	0.042	0.064	0.

Epoch 15 IOU on validataion set: 0.567713170264172

epoch: 17
lr_epoch: 0.00000, factor: 0.01000
352000	 0.008	0.002	0.028	0.018	0.172
352100	 0.028	0.019	0.062	0.042	0.218
352200	 0.003	0.001	0.037	0.015	0.198
352300	 0.023	0.007	0.174	0.119	0.432
352400	 0.005	0.006	0.065	0.043	0.206
352500	 0.003	0.002	0.054	0.034	0.210
352600	 0.016	0.009	0.148	0.097	0.251
352700	 0.005	0.004	0.123	0.074	0.270
352800	 0.010	0.003	0.051	0.064	0.224
352900	 0.008	0.007	0.085	0.088	0.296
353000	 0.007	0.013	0.113	0.078	0.497
353100	 0.006	0.010	0.083	0.109	0.245
353200	 0.005	0.006	0.086	0.086	0.217
353300	 0.005	0.004	0.103	0.091	0.316
353400	 0.005	0.006	0.073	0.037	0.204
353500	 0.007	0.011	0.020	0.029	0.421
353600	 0.006	0.005	0.086	0.058	0.246
353700	 0.024	0.027	0.056	0.036	0.228
353800	 0.006	0.006	0.051	0.041	0.256
353900	 0.047	0.031	0.114	0.043	0.323
354000	 0.005	0.004	0.028	0.029	0.211
354100	 0.013	0.010	0.120	0.105	0.272
354200	 0.007	0.007	0.104	0.074	0.278
354300	 0.005	0.008	0.019	0.0

Epoch 16 IOU on validataion set: 0.5766156190679246

epoch: 18
lr_epoch: 0.00000, factor: 0.01000
374000	 0.007	0.003	0.076	0.034	0.238
374100	 0.023	0.007	0.081	0.047	0.237
374200	 0.009	0.006	0.046	0.049	0.219
374300	 0.055	0.064	0.098	0.050	0.230
374400	 0.005	0.006	0.072	0.035	0.187
374500	 0.004	0.016	0.090	0.054	0.242
374600	 0.004	0.002	0.019	0.007	0.275
374700	 0.009	0.004	0.097	0.051	0.279
374800	 0.006	0.010	0.087	0.088	0.287
374900	 0.007	0.010	0.176	0.123	0.315
375000	 0.005	0.005	0.099	0.091	0.368
375100	 0.013	0.006	0.086	0.066	0.256
375200	 0.004	0.002	0.032	0.011	0.208
375300	 0.016	0.025	0.076	0.061	0.208
375400	 0.017	0.019	0.092	0.063	0.318
375500	 0.006	0.002	0.048	0.026	0.259
375600	 0.004	0.004	0.034	0.024	0.228
375700	 0.004	0.005	0.054	0.018	0.207
375800	 0.006	0.004	0.060	0.020	0.172
375900	 0.009	0.002	0.065	0.039	0.201
376000	 0.020	0.011	0.056	0.028	0.211
376100	 0.014	0.018	0.156	0.125	0.313
376200	 0.015	0.005	0.048	0.073	0.222
376300	 0.031	0.015	0.065	0.